In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,root_mean_squared_error
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("Data/stud.csv")

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
X = df.drop(columns=["math_score"])
y = df["math_score"]

In [5]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", cat_transformer, cat_features),
        ("StandardScaler", num_transformer, num_features),        
    ]
)

In [6]:
X = preprocessor.fit_transform(X)

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = root_mean_squared_error(true,predicted)
    r2 = r2_score(true,predicted)

    return mae,mse,rmse,r2

In [9]:
models = {
    "Linear Regression" : LinearRegression(),
    "Ridge" : Ridge(),
    "Lasso" : Lasso(),
    "SVR" : SVR(),
    "KNN" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest" : RandomForestRegressor(),
    "Adaboost" : AdaBoostRegressor(),
    "XgBoost" : XGBRegressor()
}

model_list = []
mae_list = []
mse_list = []
rmse_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train,y_train_pred)
    test_mae, test_mse, test_rmse, test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Mean Absolute Error: {:.4f}".format(train_mae))
    print("- Mean Squared Error: {:.4f}".format(train_mse))
    print("- Root Mean Squared Error: {:.4f}".format(train_rmse))
    print("- R2 Score: {:.4f}".format(train_r2))

    print('-'*35)
    
    print('Model performance for Test set')
    print("- Mean Absolute Error: {:.4f}".format(test_mae))
    print("- Mean Squared Error: {:.4f}".format(test_mse))
    print("- Root Mean Squared Error: {:.4f}".format(test_rmse))
    print("- R2 Score: {:.4f}".format(test_r2))

    mae_list.append(test_mae)
    mse_list.append(test_mse)
    rmse_list.append(test_rmse)
    r2_list.append(test_r2)

    print("="*35)
    print("\n")

Linear Regression
Model performance for Training set
- Mean Absolute Error: 4.2667
- Mean Squared Error: 28.3349
- Root Mean Squared Error: 5.3231
- R2 Score: 0.8743
-----------------------------------
Model performance for Test set
- Mean Absolute Error: 4.2148
- Mean Squared Error: 29.0952
- Root Mean Squared Error: 5.3940
- R2 Score: 0.8804


Ridge
Model performance for Training set
- Mean Absolute Error: 4.2650
- Mean Squared Error: 28.3378
- Root Mean Squared Error: 5.3233
- R2 Score: 0.8743
-----------------------------------
Model performance for Test set
- Mean Absolute Error: 4.2111
- Mean Squared Error: 29.0563
- Root Mean Squared Error: 5.3904
- R2 Score: 0.8806


Lasso
Model performance for Training set
- Mean Absolute Error: 5.2063
- Mean Squared Error: 43.4784
- Root Mean Squared Error: 6.5938
- R2 Score: 0.8071
-----------------------------------
Model performance for Test set
- Mean Absolute Error: 5.1579
- Mean Squared Error: 42.5064
- Root Mean Squared Error: 6.5197
-

In [10]:
results = pd.DataFrame(list(zip(model_list,mae_list,mse_list,rmse_list,r2_list)),columns=["Model","MAE","MSE","RMSE","R2"])
results

,Model,MAE,MSE,RMSE,R2
0,Linear Regression,4.214763,29.095170,5.393994,0.880433
1,Ridge,4.211101,29.056272,5.390387,0.880593
2,Lasso,5.157882,42.506417,6.519695,0.825320
3,SVR,5.401539,66.042005,8.126623,0.728600
4,KNN,5.621000,52.606600,7.253041,0.783813
5,Decision Tree,6.085000,59.895000,7.739186,0.753861
6,Random Forest,4.650942,36.135716,6.011299,0.851500
7,Adaboost,4.833157,37.450885,6.119713,0.846095
8,XgBoost,5.057731,41.903706,6.473307,0.827797
